<a href="https://colab.research.google.com/github/gabena98/ML_project/blob/main/ensembleNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import make_scorer, accuracy_score

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

from matplotlib import pyplot as plt

In [ ]:
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [ ]:
import keras.backend as K


def euclidean_distance_loss(y_true, y_pred):
    """
    Euclidean distance loss
    https://en.wikipedia.org/wiki/Euclidean_distance
    :param y_true: TensorFlow/Theano tensor
    :param y_pred: TensorFlow/Theano tensor of the same shape as y_true
    :return: float
    """
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

In [ ]:
#CUP
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
# load the dataset
cup_train = pd.read_csv('/content/ML-CUP22-TR_noHeader.csv', names=["x0", "x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8", "y0", "y1"]
                        )

In [ ]:
cup_train.head()

In [ ]:
cup_train.info()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(cup_train.drop(columns=['y0','y1'], axis=0),
                                                  cup_train[['y0', 'y1']],
                                                  test_size=0.4, random_state=111)

# Gridsearch

In [ ]:
def find_layer_nodes_linear(n_layers, first_layer_nodes, last_layer_nodes):
    layers = []
    
    nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
    nodes = first_layer_nodes
    for i in range(1, n_layers+1):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment
    
    return layers

In [ ]:
find_layer_nodes_linear(4,15,2)

In [ ]:
def createmodel(n_layers, first_layer_nodes, last_layer_nodes, learning_rate, momentum, activation_func, loss_func):
    model = Sequential()
    n_nodes = find_layer_nodes_linear(n_layers, first_layer_nodes, last_layer_nodes)
    for i in range(1, n_layers):
        if i==1:
            model.add(Dense(first_layer_nodes, input_dim=9, activation=activation_func))
        else:
            model.add(Dense(n_nodes[i-1], activation=activation_func))
            
    #Finally, the output layer should have a single node in binary classification
    model.add(Dense(2, activation="linear"))
    opt = tf.keras.optimizers.experimental.SGD(learning_rate, momentum=momentum)
    model.compile(optimizer=opt, loss=loss_func, metrics = ["mse"]) #note: metrics could also be 'mse'
    
    return model

##Wrap model into scikit-learn
model =  KerasRegressor(build_fn=createmodel, verbose = False)  

In [ ]:
activation_funcs = ['relu', 'elu'] 
loss_funcs = [euclidean_distance_loss]
param_grid = dict(n_layers=[4,5], first_layer_nodes = [15, 25], last_layer_nodes = [2],  learning_rate = [0.01, 0.1], momentum = [0.01, 0.1],
                  activation_func = activation_funcs, loss_func = loss_funcs, batch_size = [5,20], epochs = [250])
grid = GridSearchCV(estimator = model, param_grid = param_grid, scoring="neg_mean_absolute_percentage_error")

In [ ]:
%%time
grid.fit(X_train,y_train)

In [ ]:
print(grid.best_score_)
print(grid.best_params_)

In [ ]:
pd.DataFrame(grid.cv_results_)[['mean_test_score', 'std_test_score', 'params']].to_csv('grid_search_models_output.csv')

In [ ]:
pd.DataFrame(grid.cv_results_).columns

In [ ]:
%%time

#cross validation

fold_no = 1

# Define per-fold score containers model 1
mse_per_fold = []
loss_per_fold = []
loss_per_fold_train = []


# Define per-fold score containers model 2
mse_per_fold_2 = []
loss_per_fold_2 = []
loss_per_fold_train_2 = []

# Define per-fold score containers ensemble 
loss_per_fold_ensemble_train = [] 
loss_per_fold_ensemble_val = []

from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=False)
for train, test in kfold.split(X_train, y_train):
  print("------------")
  print("MODEL scores")
  # define the keras model 1 for cup dataset
  model_cup_1_cv = Sequential()
  model_cup_1_cv.add(Dense(15, input_shape=(9,), activation='elu'))
  model_cup_1_cv.add(Dense(11, activation='elu'))
  model_cup_1_cv.add(Dense(7, activation='elu'))
  model_cup_1_cv.add(Dense(3, activation='elu'))
  model_cup_1_cv.add(Dense(2, activation='linear'))

  # compile the keras model 1
  opt = tf.keras.optimizers.experimental.SGD(0.01, momentum=0.1)
  model_cup_1_cv.compile(optimizer=opt, loss=euclidean_distance_loss, metrics = ["mse"])

  # fit the keras model 1 on the dataset
  cv_1_cup = model_cup_1_cv.fit(X_train.iloc[train], y_train.iloc[train], validation_data=(X_train.iloc[test].values, y_train.iloc[test].values), epochs=250, batch_size=20, verbose=0)
  # Generate generalization metrics
  scores_1 = model_cup_1_cv.evaluate(X_train.iloc[test], y_train.iloc[test], verbose=0)
  print(f'Model 1 - Score for fold {fold_no}: val {model_cup_1_cv.metrics_names[0]} of {round(scores_1[0],2)}; {model_cup_1_cv.metrics_names[1]} of {round(scores_1[1],1)}')
  mse_per_fold.append(scores_1[1] * 100)
  loss_per_fold.append(scores_1[0])
  loss_per_fold_train.append(cv_1_cup.history['loss'][-1])


  # define the keras model 2 for cup dataset
  model_cup_2_cv = Sequential()
  model_cup_2_cv.add(Dense(25, input_shape=(9,), activation='elu'))
  model_cup_2_cv.add(Dense(18, activation='elu'))
  model_cup_2_cv.add(Dense(10, activation='elu'))
  model_cup_2_cv.add(Dense(2, activation='elu'))
  model_cup_2_cv.add(Dense(2, activation='linear'))

  # compile the keras model 1
  opt = tf.keras.optimizers.experimental.SGD(0.01, momentum=0.01)
  model_cup_2_cv.compile(optimizer=opt, loss=euclidean_distance_loss, metrics = ["mse"])

  # fit the keras model 1 on the dataset
  cv_2_cup = model_cup_2_cv.fit(X_train.iloc[train], y_train.iloc[train], validation_data=(X_train.iloc[test].values, y_train.iloc[test].values), epochs=250, batch_size=20, verbose=0)
  # Generate generalization metrics
  scores = model_cup_2_cv.evaluate(X_train.iloc[test], y_train.iloc[test], verbose=0)
  print(f'Model 2 - Score for fold {fold_no}: val {model_cup_2_cv.metrics_names[0]} of {round(scores[0],2)}; {model_cup_2_cv.metrics_names[1]} of {round(scores[1],1)}')
  mse_per_fold_2.append(scores[1] * 100)
  loss_per_fold_2.append(scores[0])
  loss_per_fold_train_2.append(cv_2_cup.history['loss'][-1])


  print("-----Ensemble scores------")

  #averaging models making an ensemble
  models=[model_cup_1_cv, model_cup_2_cv]
  yhats = [model.predict(X_train.iloc[train], verbose=0) for model in models]
  averaged_predictions = (yhats[1]+yhats[0])/2
  euclidean_loss_ensemble = np.mean(euclidean_distance_loss(averaged_predictions, y_train.iloc[train].values))
  print(f"Fold {fold_no}: MEE of predictions on 4 folds--> {round(euclidean_loss_ensemble,2)}")
  loss_per_fold_ensemble_train.append(round(euclidean_loss_ensemble,2))

  #averaging models making an ensemble
  models=[model_cup_1_cv, model_cup_2_cv]
  yhats = [model.predict(X_train.iloc[test], verbose=0) for model in models]
  averaged_predictions = (yhats[1]+yhats[0])/2
  euclidean_loss_ensemble = np.mean(euclidean_distance_loss(averaged_predictions, y_train.iloc[test].values))
  print(f"Fold {fold_no}: MEE of predictions on 1 fold--> {round(euclidean_loss_ensemble,2)}")
  loss_per_fold_ensemble_val.append(round(euclidean_loss_ensemble,2))





  # Increase fold number
  fold_no = fold_no + 1

In [ ]:
print("MODEL 1 - CV")
avg_mee_cv_model_1 = round(sum(loss_per_fold) / len(loss_per_fold), 2)
avg_mee_cv_model_1_train = round(sum(loss_per_fold_train) / len(loss_per_fold_train), 2)
print(f"The average euclidean error for the CV of model 1 is {avg_mee_cv_model_1} (for validation split)")
print(f"The average euclidean error for the CV of model 1 is {avg_mee_cv_model_1_train} (for train split)")

print("MODEL 2 - CV")
avg_mee_cv_model_2 = round(sum(loss_per_fold_2) / len(loss_per_fold_2), 2)
avg_mee_cv_model_2_train = round(sum(loss_per_fold_train_2) / len(loss_per_fold_train_2), 2)
print(f"The average euclidean error for the CV of model 2 is {avg_mee_cv_model_2} (for validation split)")
print(f"The average euclidean error for the CV of model 2 is {avg_mee_cv_model_2_train} (for train split)")

print("Ensemble - CV")
avg_mee_cv_ensemble = round(sum(loss_per_fold_ensemble_val) / len(loss_per_fold_ensemble_val), 2)
avg_mee_cv_ensemble_train = round(sum(loss_per_fold_ensemble_train) / len(loss_per_fold_ensemble_train), 2)
print(f"The average euclidean error for the CV of the ensemble is {avg_mee_cv_ensemble} (for validation split)")
print(f"The average euclidean error for the CV of the ensemble is {avg_mee_cv_ensemble_train} (for train split)")

#Create ensemble with 2 different models with params from gridsearch

In [ ]:
%%time
...
# define the keras model 1 for cup dataset
model_cup_1 = Sequential()
model_cup_1.add(Dense(15, input_shape=(9,), activation='elu'))
model_cup_1.add(Dense(11, activation='elu'))
model_cup_1.add(Dense(7, activation='elu'))
model_cup_1.add(Dense(3, activation='elu'))
model_cup_1.add(Dense(2, activation='linear'))

# compile the keras model 1
opt = tf.keras.optimizers.experimental.SGD(0.01, momentum=0.1)
model_cup_1.compile(optimizer=opt, loss=euclidean_distance_loss, metrics = ["mse"])

# fit the keras model 1 on the dataset
history_1_cup = model_cup_1.fit(X_train, y_train,validation_data=(X_val.values, y_val.values) , epochs=250, batch_size=20)

...

In [ ]:
plt.plot(history_1_cup.history['loss'])
plt.plot(history_1_cup.history['val_loss'], linestyle="dashed")
plt.axis([0, 250, 1.3, 2.25])
plt.title('model 1 loss - CUP')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('cup_member_1.png')
plt.show()

In [ ]:
%%time
...
# define the keras model 2 for cup dataset
model_cup_2 = Sequential()
model_cup_2.add(Dense(25, input_shape=(9,), activation='elu'))
model_cup_2.add(Dense(18, activation='elu'))
model_cup_2.add(Dense(10, activation='elu'))
model_cup_2.add(Dense(2, activation='elu'))
model_cup_2.add(Dense(2, activation='linear'))

# compile the keras model 1
opt = tf.keras.optimizers.experimental.SGD(0.01, momentum=0.01)
model_cup_2.compile(optimizer=opt, loss=euclidean_distance_loss, metrics = ["mse"])

# fit the keras model 1 on the dataset
history_2_cup = model_cup_2.fit(X_train, y_train, validation_data=(X_val.values, y_val.values), epochs=250, batch_size=20)
...

In [ ]:
plt.plot(history_1_cup.history['loss'])
plt.plot(history_1_cup.history['val_loss'], linestyle="dashed")
plt.axis([0, 250, 1.3, 3.5])
plt.title('model 2 loss - CUP')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('cup_member_2.png')
plt.show()

In [ ]:
#averaging models making an ensemble
models=[model_cup_1, model_cup_2]

In [ ]:
yhats = [model.predict(X_val, verbose=0) for model in models]
averaged_predictions = (yhats[1]+yhats[0])/2
euclidean_loss_ensemble = np.mean(euclidean_distance_loss(averaged_predictions, y_val.values))
euclidean_loss_model_1_test = np.mean(euclidean_distance_loss(yhats[0], y_val.values))
euclidean_loss_model_2_test = np.mean(euclidean_distance_loss(yhats[1], y_val.values))
print(f"Euclidean loss ensemble {round(euclidean_loss_ensemble,2)}")
print(f"Euclidean loss model 1 {round(euclidean_loss_model_1_test,2)}")
print(f"Euclidean loss model 2 {round(euclidean_loss_model_2_test,2)}")

#CUP final prediction

In [ ]:
# load the dataset
cup_train = pd.read_csv('/content/ML-CUP22-TS_noHeader.csv', names=["x0", "x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8", "y0", "y1"]
                        )

In [ ]:
yhats_cup_final = [model.predict(X_val) for model in models]
averaged_predictions_cup_final = (yhats_cup_final[1]+yhats_cup_final[0])/2

In [ ]:
pd.DataFrame(averaged_predictions_cup_final, columns=["y0", "y1"]).to_csv("averaged_predictions_cup_final_ensemble.csv", index=False)